# Dynamics via acceleration maps

- `medium` dataset has `n_stats = 500`
- `large` dataset has `n_stats = 250`
- `varying target area` dataset has `n_stats = 250`



In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from analysis import analysis

L_box = 50
mu_factor = 6  # conversion to microns
min_factor = 8  # conversion to minutes

#### Load grid by grid
- Load `fulltake_gid*.pkl`, which is all of the runs for a config

In [ ]:
from glob import glob

files = glob("data/new mp/fulltake_gid1*.pkl")
data = []

for file in files:
    df = pd.read_pickle(file)

    # INCREASE TIME INTERVAL BTW RECORDS
    # df = df.iloc[::2]

    df.x *= mu_factor
    df.y *= mu_factor
    data.append(df)

print(f"Loaded {len(data)} configurations.")
[
    print(
        f"\t - Grid {gid}: {len(df)} total data points | {len(df.rid.unique())} runs | {len(df[df.rid==0])} data points / run"
    )
    for gid, df in zip(range(8), data)
]
display(data[0])


### Position dist

In [ ]:
plt.rc('axes', titlesize=8)
analysis.position_dist(data, mu_factor)

### Dropping configs that don't cross

In [ ]:
filtered = []
for df in data:
    if df.gid.iloc[0] in [103, 105, 107]:
        filtered.append(df)
data = filtered

### Hopping times

In [ ]:
fig, axs = plt.subplots(3, 3, figsize=(8, 6), dpi=200)
plt.rc('axes', titlesize=12)

x, y = np.meshgrid([0, 1, 2], [0, 1, 2])

# in microns
d = 73 / 2
xL = (L_box / 2) * 6 - d
xR = (L_box / 2) * 6 + d
buffer = 38 / 2

for i, j, df in zip(x.flatten(), y.flatten(), data):
    beta = df.beta.iloc[0]
    tau = df.tau.iloc[0]
    D = df.D.iloc[0]
    axs[i, j].set_title(rf"$\beta$ = {beta}, $\tau$ = {tau}, $D$ = {D}", y=1.1)

    hop_times = []
    for rid, d in df.groupby("rid"):
        for t in analysis.get_hopping_times(d, xL, xR, buffer):
            hop_times.append(t)

    if len(hop_times) == 0:
        continue

    axs[i, j].hist(hop_times, color="red", alpha=0.5, density=True)
    axs[i, j].set_xlabel("Hopping time (hr)", fontsize=14)

axs[2, 2].remove()
plt.tight_layout()
plt.subplots_adjust(hspace=2, wspace=2)
plt.show()


## Map bin statistics

In [ ]:
fig1, axs1 = plt.subplots(3, 3, figsize=(8, 6), dpi=200)
fig2, axs2 = plt.subplots(3, 3, figsize=(8, 6), dpi=200)

x, y = np.meshgrid([0, 1, 2], [0, 1, 2])
global_x_v_a = []

for i, j, df_gid in zip(x.flatten(), y.flatten(), data):
    beta = df_gid.beta.iloc[0]
    tau = df_gid.tau.iloc[0]
    D = df_gid.D.iloc[0]
    axs1[i, j].set_title(rf"$\beta$ = {beta}, $\tau$ = {tau}, $D$ = {D}", y=1.1)
    axs2[i, j].set_title(rf"$\beta$ = {beta}, $\tau$ = {tau}, $D$ = {D}", y=1.1)

    # x, v, a values for this entire config
    grid_x_v_a = []

    # compute speed and acc for each run
    for rid, df_rid in df_gid.groupby("rid"):
        x_v_a = analysis.calc_v_a_from_position(df_rid.x, df_rid["time[hr]"])
        x_v_a["gid"] = df_rid.gid.iloc[0]
        x_v_a["rid"] = df_rid.rid.iloc[0]
        grid_x_v_a.append(x_v_a)

    # consider the entire config to set bounds of acc field
    grid_x_v_a = pd.concat(grid_x_v_a)
    bounds = grid_x_v_a.agg(["min", "max"])
    xmin, xmax = bounds["x"]
    vmin, vmax = bounds["v"]

    # with a 112mu mp, this gives bins of dim 3.5mu x 3.5mu
    nbins = 32
    dx = (xmax - xmin) / nbins
    dv = (vmax - vmin) / nbins

    x1 = ("x", xmin, dx)
    x2 = ("v", vmin, dv)

    analysis.get_bin_indices(grid_x_v_a, nbins, x1, x2)

    bin_counts = grid_x_v_a.groupby(["x_bin", "v_bin"]).size().to_numpy()
    bin_stds = grid_x_v_a.groupby(["x_bin", "v_bin"]).a.std().to_numpy()
    axs1[i, j].hist(bin_counts, bins=30, color="black", alpha=0.8)
    axs2[i, j].hist(bin_stds/np.sqrt(bin_counts), bins=30, color="orange", alpha=0.8)

    if i == 2:
        axs1[i, j].set_xlabel("Bin counts")
        axs2[i, j].set_xlabel(r"std err of mean ($\mu$m/hr$^2$)")

    global_x_v_a.append(grid_x_v_a)

axs1[2, 2].remove()
axs2[2, 2].remove()

fig1.tight_layout()
fig1.subplots_adjust(hspace=1.5, wspace=.5)

fig2.tight_layout()
fig2.subplots_adjust(hspace=1.5, wspace=.5)

plt.show()

## Acceleration maps and flow lines

In [ ]:
from importlib import reload
reload(analysis)

In [ ]:
plt.rc("axes", titlesize=10)

fig1, axs1 = plt.subplots(3, 2, figsize=(4, 5), dpi=200)
fig2, axs2 = plt.subplots(3, 2, figsize=(4, 5), dpi=200)

x, y = np.meshgrid([0, 1, 2], [0, 1])

for i, j, df_gid, my_grid in zip(x.flatten(), y.flatten(), data, global_x_v_a):
    beta = df_gid.beta.iloc[0]
    tau = df_gid.tau.iloc[0]
    D = df_gid.D.iloc[0]

    axs1[i, j].set_title(rf"$\beta$ = {beta}, $\tau$ = {tau}, $D$ = {D}", y=1.1)
    axs2[i, j].set_title(rf"$\beta$ = {beta}, $\tau$ = {tau}, $D$ = {D}", y=1.1)

    bounds = my_grid.agg(["min", "max"])
    xmin, xmax = bounds["x"]
    vmin, vmax = bounds["v"]

    acc_map = analysis.map_a(my_grid, nbins, "x_bin", "v_bin", min_pts=2)
    analysis.map_imshow(acc_map, axs1[i, j], (xmin, xmax, vmin, vmax), nbins)
    analysis.streamplot(
        acc_map, axs2[i, j], (xmin, xmax, vmin, vmax), nbins, init_cond=True, n_skip=5, one_side=None
    )

    axs1[i, j].axis("off")
    axs2[i, j].axis("off")

[ax.remove() for ax in axs1[1:, 1]]
fig1.tight_layout()
fig1.subplots_adjust(hspace=1)

[ax.remove() for ax in axs2[1:, 1]]
fig2.tight_layout()
fig2.subplots_adjust(hspace=1)

plt.show()


# Debugging purposes

In [ ]:
from substrate.substrates import Substrate

sub = Substrate(200, 50, 0.2)
mp = sub.two_state_sub(bridge_width=17)

plt.imshow(mp, extent=[0, 300, 0, 300], cmap="Blues_r", alpha=0.5)
plt.colorbar()
plt.contour(mp, levels=[0.5], extent=[0, 300, 0, 300], linewidths=[2])
plt.ylim([100, 200])
plt.xlim([75, 225])

plt.show()



In [ ]:
# PLOT FLOW LINES
X, Y = np.meshgrid(np.linspace(xmin, xmax, nbins), np.linspace(vmin, vmax, nbins))

fig, ax = plt.subplots(1, 1)
fig_temp, ax_temp = plt.subplots(1, 1)

ax.streamplot(
    X,
    Y,
    Y,
    acc_map,
    linewidth=2,
    integration_direction="forward",
    color="gainsboro",
    density=2,
)

x, y = X[0][np.where(acc_map != np.nan)[1]], Y[:, 0][np.where(acc_map != np.nan)[0]]

x, y = x[100:350], y[100:350]

for xx, yy in zip(x, y):
    stream = ax_temp.streamplot(
        X,
        Y,
        Y,
        acc_map,
        start_points=[[xx, yy]],
        integration_direction="forward",
        broken_streamlines=False
    )
    traj = np.array(stream.lines.get_segments())
    
    if len(traj) == 0:
        continue

    flag = traj[-1][-1][0] < 150
    if flag:
        color="red"
    else:
        color="blue"
    
    ax.streamplot(
        X,
        Y,
        Y,
        acc_map,
        linewidth=0.5,
        start_points=[[xx, yy]],
        integration_direction="forward",
        color=color,
        broken_streamlines=False
    )

plt.close(fig_temp)
plt.show()